### **Análise Estratégica de Clientes da KAMI CO Usando a Classificação RFV**

A classificação **RFV** (Recência, Frequência e Valor) não é apenas uma metodologia; é uma ferramenta transformadora para a **KAMI CO**, uma empresa proeminente no segmento de beleza que opera com destaque na indústria, distribuição e comércio eletrônico. Ao mergulhar profundamente nos dados de nossos clientes por meio do RFV, temos a capacidade única de:

- **Identificar os clientes que mais agregam valor**
- **Criar estratégias de marketing finamente sintonizadas com os padrões de compra**
- **Otimizar nosso inventário e recursos com precisão sem precedentes.**

Este documento visa descortinar o potencial oculto em nossos dados, garantindo que a **KAMI CO** se posicione na vanguarda do mercado de beleza, sempre ágil e alinhada às necessidades de nossos clientes.

#### **Descrição do Conjunto de Dados**

O conjunto de dados é composto por todos os clientes que fizeram pelo menos uma compra desde janeiro de 2022 em algumas das empresas listadas abaixo:
| cod_empresa | nome_fantasia                            |
|-------------|------------------------------------------|
| 1	          | KAMI COSMÉTICOS                          |
| 2	          | NEW HAUSS SOLUCOES PARA SALOES DE BELEZA |
| 3	          | MOVEMENT                                 |
| 4	          | ENERGY COSMETICOS LTDA                   |
| 5	          | HAIRPRO COMERCIO DE COSMETICOS LTDA      |
| 6	          | SOUTH COSMETICOS                         |
| 9	          | THE BEST HAIR COMERCIO E REPRESENTACAO   |
| 10	        | 3MKO COSMETICOS LTDA MATRIZ              |
| 11	        | 3MKO COSMETICOS LTDA FILIAL SP           |
| 12	        | 3MKO COSMETICOS LTDA FILIAL ES           |
| 13	        | MOVEMENT RJ                              |
| 14	        | 3MKO COSMETICOS LTDA FILIAL PR           |
| 15	        | MOVEMENT MT                              |
| 16	        | MOVEMENT RS                              |



In [1]:
# Atualiza Dataset de Classificação de RFV
from kami_uno_database import get_dataframe_from_sql
import numpy as np
RFV_CLASS_NUM_COLS = {
  'cod_cliente': np.int64,
  'recencia': np.int64,
  'dias_ultima_compra': np.int64,
  'qtd_compras_ultimo_ano': np.int64,
  'frequencia': np.int64,
  'ticket_medio': np.float64,
}
rfv_class_df = get_dataframe_from_sql(
  sql_query='SELECT * FROM vw_rfv_classification',            
  cols_types=RFV_CLASS_NUM_COLS,
)
clients_to_remove = [34581, 700]
rfv_class_df = rfv_class_df[rfv_class_df['ticket_medio'] > 0]
rfv_class_df = rfv_class_df.sort_values(by='ticket_medio', ascending=False)
rfv_class_df = rfv_class_df[~rfv_class_df['cod_cliente'].isin(clients_to_remove)]

2023-10-24 02:35:40 - [database] [INFO]: Calling get_dataframe_from_sql
2023-10-24 02:35:40 - [database] [INFO]: Calling get_dataframe_from_sql_query
2023-10-24 02:35:40 - [database] [INFO]: Calling create_and_connect_engine
2023-10-24 02:35:40 - [database] [INFO]: Finished Calling create_and_connect_engine
2023-10-24 02:35:40 - [database] [INFO]: Execution of create_and_connect_engine took 0.204.
2023-10-24 02:35:44 - [database] [INFO]: Finished Calling get_dataframe_from_sql_query
2023-10-24 02:35:44 - [database] [INFO]: Execution of get_dataframe_from_sql_query took 3.940.
2023-10-24 02:35:44 - [database] [INFO]: Finished Calling get_dataframe_from_sql
2023-10-24 02:35:44 - [database] [INFO]: Execution of get_dataframe_from_sql took 3.942.


#### **Recência: Primeira Dimensão Central da RFV**

A **Recência** mede o tempo desde a última transação de um cliente. É um indicador crucial: quanto mais recente a compra, maior a probabilidade de o cliente retornar. A fórmula para calcular a recência é:
$$
\text{Recência} = \text{Data Atual} - \text{Data da Última Compra}
$$
Por exemplo, se um cliente comprou um produto da **KAMI CO** no dia 10 de janeiro e hoje é dia 20, sua recência é de 10 dias.
Adotamos uma pontuação R, variando de 1 a 5, para classificar a recência. A pontuação se baseia na seguinte divisão:

| Pontuação R | Última Compra  |
|-------------|----------------|
| **5**       | Até 30 dias    |
| **4**       | 31 a 75 dias   |
| **3**       | 76 a 120 dias  |
| **2**       | 121 a 165 dias |
| **1**       | 166 a 180 dias |

Esta pontuação nos permite segmentar nossos clientes de acordo com sua recente interação com a **KAMI CO**, otimizando nossas estratégias de marketing.

In [2]:
# Análise de Recência
import plotly.graph_objects as go
import plotly.subplots as sp

def show_recency_legend():
  table_headers = ["Recência", "Última Compra"]
  table_values = [
    ["5", "4", "3", "2", "1"],
    ["30 dias ou menos", "31 a 60 dias", "61 a 90 dias", "91 a 150 dias", "180 dias ou mais"]
  ]
  rows_colors = ["#FF6100", "#FF952B", "#FFB366", "#FFD1B3", "#FFB3B3"]
  header_colors = ["#FF4100", "#FF4100"]
  tabl_fig = go.Figure(go.Table(
      header=dict(values=table_headers, 
                  fill_color=header_colors,
                  font=dict(color="white", size=16),
                  height=32),
      cells=dict(values=table_values, 
                 fill_color=[rows_colors, rows_colors],
                 font=dict(color="black", size=14),
                 height=32)))
  tabl_fig.update_layout(
     title="Tabela de Referência para Pontuação de Recência",
     title_x=0.5, title_font_size=20,
     showlegend=False, width=600, height=400
  )
  tabl_fig.show()

  # Recency Graphs
def show_recency_graphs():  
  abs_counts = rfv_class_df['recencia'].value_counts().sort_index()
  perc_counts = (rfv_class_df['recencia'].value_counts(normalize=True) * 100).sort_index()
  orange_colors = ["#FFD1B3", "#FFB366", "#FF952B", "#FF7A00", "#FF6100"]
  red_colors = ["#FFB3B3", "#FF6666", "#FF392B", "#FF1A00", "#FF0000"]
  fig = sp.make_subplots(rows=1, cols=2,
                         subplot_titles=("Clientes por Pontuação de Recência",
                                         "% de Clientes por Recência"))
  
  fig.add_trace(go.Bar(x=abs_counts, 
                       y=abs_counts.index.astype(str), 
                       orientation='h',
                       marker=dict(color=orange_colors)), 
                row=1, col=1)
  fig.update_xaxes(title_text="Número de Clientes (Unidades)", row=1, col=1)
  fig.update_yaxes(title_text="Pontuação de Recência", row=1, col=1)

  fig.add_trace(go.Bar(x=perc_counts.index.astype(str), 
                       y=perc_counts, 
                       marker=dict(color=red_colors)), 
                row=1, col=2)
  fig.update_xaxes(title_text="Pontuação de Recência", row=1, col=2)
  fig.update_yaxes(title_text="% de Clientes (%)", row=1, col=2)
  
  # Adjust title font size and layout
  for i in fig['layout']['annotations']:
      i['font']['size'] = 20
  
  fig.update_layout(showlegend=False, height=400)
  fig.show()
def show_recency_indicators():  
  orange_colors = ["#FFD1B3", "#FFB366", "#FF952B", "#FF7A00", "#FF6100"]  
  avg_days_last_purchase = rfv_class_df['dias_ultima_compra'].mean()
  avg_recency = rfv_class_df['recencia'].mean()
  target_days = 30  
  
  avg_last_purchase_fig = go.Figure(go.Indicator(
    mode='number+delta',
    title={
      'text': f"Média de Dias Desde A Última Compra",
      'font': {'size': 22, 'color':orange_colors[2]}
    },
    value = avg_days_last_purchase,
    number = {'font': {'size': 50, 'color':orange_colors[4]}},
    delta={
      'relative': True,
      'valueformat': '.1%',
      'reference': target_days,
      'increasing': {'color': "red", 'symbol': "▼"},
      'decreasing': {'color': "green", 'symbol': "▲"},
      'position': "bottom",      
      'suffix': f" abaixo da meta" if avg_days_last_purchase > target_days else ' acima da meta'
    },    
    domain = {'row': 0, 'column': 0}
  ))
  avg_recency_fig = go.Figure(go.Indicator(
        mode='gauge+number',
        value=avg_recency,
        gauge={
            'axis': {'range': [None, 5], 'tickvals': [1,2,3,4,5], 'ticktext': ["1", "2", "3", "4", "5"]},
            'bar': {'color': orange_colors[2]},
            'steps': [
                {'range': [0, 1], 'color': orange_colors[0]},
                {'range': [1, 2], 'color': orange_colors[1]},
                {'range': [2, 3], 'color': orange_colors[2]},
                {'range': [3, 4], 'color': orange_colors[3]},
                {'range': [4, 5], 'color': orange_colors[4]},
            ],
        },
        title={
            'text': "Média de Recência",
            'font': {'size': 16, 'color': orange_colors[2]}
        },
        number={'font': {'size': 50, 'color': orange_colors[4]}}
    )) 
  fig = go.Figure()
  fig.add_trace(avg_last_purchase_fig.data[0])
  fig.add_trace(avg_recency_fig.data[0])
  fig.update_layout(grid = {'rows': 1, 'columns': 2, 'pattern': "independent"})
  fig.show()
def show_recency_indicators():
    orange_colors = ["#FFD1B3", "#FFB366", "#FF952B", "#FF7A00", "#FF6100"]
    avg_days_last_purchase = rfv_class_df['dias_ultima_compra'].mean()
    avg_recency = rfv_class_df['recencia'].mean()
    target_days = 30
   
    avg_last_purchase_trace = go.Indicator(
        mode='number+delta',
        title={
            'text': f"Média de Dias Desde A Última Compra",
            'font': {'size': 22, 'color':orange_colors[2]}
        },
        value=avg_days_last_purchase,
        number={'font': {'size': 50, 'color':orange_colors[4]}},
        delta={
            'relative': True,
            'valueformat': '.1%',
            'reference': target_days,
            'increasing': {'color': "red", 'symbol': "▼"},
            'decreasing': {'color': "green", 'symbol': "▲"},
            'position': "bottom",
            'suffix': f" abaixo da meta de 30 dias" if avg_days_last_purchase > target_days else ' acima da meta de 30 dias'
        },
        domain={'row': 0, 'column': 0}
    )

    avg_recency_trace = go.Indicator(
        mode='gauge+number',
        value=avg_recency,
        gauge={
            'shape': "angular",
            'bar': {'color': "darkred"},
            'axis': {'range': [None, 5], 'tickvals': [1,2,3,4,5], 'ticktext': ["1", "2", "3", "4", "5"]},
            'steps': [
                {'range': [0, 1], 'color': orange_colors[0]},
                {'range': [1, 2], 'color': orange_colors[1]},
                {'range': [2, 3], 'color': orange_colors[2]},
                {'range': [3, 4], 'color': orange_colors[3]},
                {'range': [4, 5], 'color': orange_colors[4]},
            ],
            'threshold': {
                'line': {'color': "darkred", 'width': 6},
                'thickness': 0.75,
                'value': avg_recency
            }
        },
        title={
            'text': "Média de Recência",
            'font': {'size': 22, 'color': orange_colors[2]}
        },
        number={'font': {'size': 80, 'color': orange_colors[4]}},
        domain={'row': 0, 'column': 1}
    )

    fig = go.Figure()

    fig.add_trace(avg_last_purchase_trace)
    fig.add_trace(avg_recency_trace)
    fig.update_layout(grid={'rows': 1, 'columns': 2, 'pattern': "independent"})

    fig.show()
show_recency_legend()
show_recency_graphs()
show_recency_indicators()

#### **Frequência: Segunda Dimensão Essencial da RFV**

**Frequência** indica quantas vezes um cliente comprou em um determinado período. Ela nos mostra a regularidade com que os clientes interagem com a marca. Simplificando, na análise RFV, avaliamos a frequência de compras do ano de 2022 da seguinte forma:

$$
\text{Frequência} = \frac{\text{Número Total de Compras em 2022}}{12}
$$

Por exemplo, um cliente que realizou 24 compras na **KAMI CO** em 2022 tem uma frequência média de 2 compras por mês.
Para segmentar os clientes, utilizamos uma pontuação F, de 1 a 5, com base na frequência mensal média:

| Pontuação F | Média de Compras / Mês |
|-------------|------------------------|
| **5**       | >= 2                   |
| **4**       | >= 1,5 e < 1,9         |
| **3**       | >= 1 e < 1,4           |
| **2**       | >= 0,5 e < 0,9         |
| **1**       | < de 0,5               |

Essa segmentação ajuda a identificar clientes frequentes, otimizando estratégias direcionadas para a **KAMI CO**.


In [3]:
# Análise de Frequência
import plotly.graph_objects as go
import numpy as np

rfv_class_df['media_mensal_compras'] = rfv_class_df['qtd_compras_ultimo_ano'] / 12
highest_avg = rfv_class_df[rfv_class_df['media_mensal_compras'] == rfv_class_df['media_mensal_compras'].max()].iloc[0]
general_avg = rfv_class_df['media_mensal_compras'].mean()

def show_frequency_analisys_indicators():
  top1_costumer_frequency_fig = go.Indicator(
    mode="number",
    title={
      'text': f"<span style='font-size:100%; color: steelblue;'>Total de Compras em 2022</span><br>",
      'font': {'size': 20}
    },
    value=highest_avg['qtd_compras_ultimo_ano'],
    number={'font': {'size': 24, 'color':"deepskyblue"}},
    domain={'row': 0, 'column': 0}
  )
  top1_costumer_avg_montlhy_purchase_fig = go.Indicator(
      mode="number",
    title={
      'text': f"<span style='font-size:100%; color: steelblue;'>Média Mensal de Compras em 2022</span><br>",
      'font': {'size': 20}
    },
    value=highest_avg['media_mensal_compras'],
    number={'font': {'size': 24, 'color':"deepskyblue"}},
    domain={'row': 0, 'column': 1}
  )
  
  fig = go.Figure()
  fig.add_trace(top1_costumer_frequency_fig)
  fig.add_trace(top1_costumer_avg_montlhy_purchase_fig)
  fig.update_layout(
    title={
      'text': f"Cliente Mais Frequente em 2022<br>{highest_avg['nome_cliente']}<br>{highest_avg['razao_social']}",
      'y':0.9,
      'x':0.5,
      'xanchor': 'center',
      'yanchor': 'top'
    },
    grid={'rows': 1, 'columns': 2, 'pattern': "independent"},
    height=250,
    width=800
  )
  fig.show()

def show_top20_customers_by_frequency():
  top_20 = rfv_class_df.nlargest(20, 'qtd_compras_ultimo_ano')
  top_20 = top_20.sort_values(by='qtd_compras_ultimo_ano', ascending=True)
  
  bar_fig = go.Figure(go.Bar(
    y=top_20['nome_cliente'],
    x=top_20['qtd_compras_ultimo_ano'],
    orientation='h',
    marker={
        'color': top_20['qtd_compras_ultimo_ano'],
        'colorscale': 'blues'
    },
    text=top_20['qtd_compras_ultimo_ano'],
    textposition='outside'
  ))
  
  avg_purchases = top_20['qtd_compras_ultimo_ano'].mean()
  frequency_5 = avg_purchases / 12
  freq_5_annotation_text = (
    f"Média de Compras dos 20 Clientes: <b>{avg_purchases:.2f}</b><br>"
    f"12 Meses<br>"
    f"Frequency 5 (corte): <b>{frequency_5:.2f}</b>"
  )

  bar_fig.update_layout(
    title="Top 20 Clientes por Frequência de Compras em 2022",
    xaxis_title="Quantidade de Compras",
    yaxis_title="Cliente",
    showlegend=False,
    height=900,
    annotations=[
      {
        'x': avg_purchases,
        'y': top_20['nome_cliente'].iloc[-1],
        'xref': 'x',
        'yref': 'y',
        'text': f"Média: {avg_purchases:.2f}",
        'showarrow': True,
        'arrowhead': 4,
        'ax': 0,
        'ay': -40,
        'font': {'color': 'red', 'size': 18}
      },      
      {
        'x': 0,
        'y': -4,
        'xref': 'paper',
        'yref': 'y',
        'text': freq_5_annotation_text,
        'showarrow': False,
        'align': 'left',
        'font': {'color': 'black', 'size': 14}
      }
    ]
  )
  bar_fig.show()

def show_frequency_legend():    
    step = (6 - 1) / 4
    bins = list(np.arange(1, 6 + step, step)[::-1])    
    freq_ranges = [
        f"{bins[i]:.2f} to {bins[i+1]:.2f}" for i in range(len(bins)-1)
    ]
    freq_ranges.append("< 1.00")
    scores = list(range(5, 0, -1))
    colors = ['#08306b', '#08519c', '#2171b5', '#4292c6', '#6baed6']
    fig = go.Figure(data=[go.Table(
        header=dict(values=['Frequência', 'Média Mensal de Compras'],
                    fill_color='darkblue',
                    font=dict(color='white', size=16),
                  height=32),
        cells=dict(values=[scores, freq_ranges],
                   fill_color=[colors, colors],
                   align=['center', 'center'],
                   font=dict(color='black', size=14),
                  height=32)
    )])

    fig.update_layout(
      title="Tabela de Referência para Pontuação de Frequência",
      title_x=0.5,
      title_font_size=20,
      showlegend=False,
      width=800,
      height=400
    )
    fig.show()

def show_frequency_graphs():    
  abs_counts = rfv_class_df['frequencia'].value_counts().sort_index()
  perc_counts = (rfv_class_df['frequencia'].value_counts(normalize=True) * 100).sort_index()
  fig = sp.make_subplots(
    rows=1, cols=2,
    subplot_titles=(
      "Clientes por Pontuação de Frequência",
      "% de Clientes por Frequência")
  )
  fig.add_trace(go.Bar(x=abs_counts, 
                       y=abs_counts.index.astype(str), 
                       orientation='h',
                       marker=dict(color=abs_counts.values, colorscale='purples')), 
                row=1, col=1)
  fig.update_xaxes(title_text="Número de Clientes (Unidades)", row=1, col=1)
  fig.update_yaxes(title_text="Pontuação de Frequência", row=1, col=1)
  fig.add_trace(go.Bar(x=perc_counts.index.astype(str), 
                       y=perc_counts, 
                       marker=dict(color=perc_counts.values, colorscale='blues')), 
                row=1, col=2)
  fig.update_xaxes(title_text="Pontuação de Frequência", row=1, col=2)
  fig.update_yaxes(title_text="% de Clientes (%)", row=1, col=2)  
  for i in fig['layout']['annotations']:
      i['font']['size'] = 20  
  fig.update_layout(showlegend=False, height=400)
  fig.show()

def show_frequency_indicators():
    orange_colors = ["#FFD1B3", "#FFB366", "#FF952B", "#FF7A00", "#FF6100"]
    last_year_purchases = rfv_class_df['qtd_compras_ultimo_ano'].mean()
    avg_frequency = rfv_class_df['frequencia'].mean()
    target_montlhy_purchase = 36
   
    avg_last_purchase_trace = go.Indicator(
        mode='number+delta',
        title={
            'text': f"Média Compras no Último Ano",
            'font': {'size': 22, 'color':orange_colors[2]}
        },
        value=last_year_purchases,
        number={'font': {'size': 50, 'color':orange_colors[4]}},
        delta={
            'relative': True,
            'valueformat': '.1%',
            'reference': target_montlhy_purchase,            
            'position': "bottom",
            'suffix': f" abaixo da meta de 36 compras anuais" if last_year_purchases < target_montlhy_purchase else ' acima da meta de 36 compras anuais'
        },
        domain={'row': 0, 'column': 0}
    )

    avg_frequency_trace = go.Indicator(
        mode='gauge+number',
        value=avg_frequency,
        gauge={
            'shape': "angular",
            'bar': {'color': "darkred"},
            'axis': {'range': [None, 5], 'tickvals': [1,2,3,4,5], 'ticktext': ["1", "2", "3", "4", "5"]},
            'steps': [
                {'range': [0, 1], 'color': orange_colors[0]},
                {'range': [1, 2], 'color': orange_colors[1]},
                {'range': [2, 3], 'color': orange_colors[2]},
                {'range': [3, 4], 'color': orange_colors[3]},
                {'range': [4, 5], 'color': orange_colors[4]},
            ],
            'threshold': {
                'line': {'color': "darkred", 'width': 6},
                'thickness': 0.75,
                'value': avg_frequency
            }
        },
        title={
            'text': "Média de Frequência",
            'font': {'size': 22, 'color': orange_colors[2]}
        },
        number={'font': {'size': 80, 'color': orange_colors[4]}},
        domain={'row': 0, 'column': 1}
    )

    fig = go.Figure()

    fig.add_trace(avg_last_purchase_trace)
    fig.add_trace(avg_frequency_trace)
    fig.update_layout(grid={'rows': 1, 'columns': 2, 'pattern': "independent"}, height=600)

    fig.show()

show_frequency_analisys_indicators()
show_top20_customers_by_frequency()
show_frequency_legend()
show_frequency_graphs()
show_frequency_indicators()

#### **Valor: Terceira Dimensão Fundamental da RFV**

**Valor** reflete quanto um cliente gasta em média por compra. No contexto da análise RFV, avaliamos o ticket médio das compras de 2022, com a fórmula:

$$
\text{Ticket Médio} = \frac{\text{Gasto Total do Cliente em 2022}}{\text{Número Total de Compras em 2022}}
$$

Se um cliente desembolsou R$2400 na **KAMI CO** em 2022 com 12 compras, seu ticket médio é R$200.
Para segmentar os clientes, utilizamos uma pontuação V, de 1 a 5, baseada no ticket médio do ano anterior:


| Pontuação V | Faixa de Ticket Médio |
|-------------|-----------------------|
| **5** | Top 20% dos clientes  |
| **4** | Top 40% dos clientes  |
| **3** | Top 60% dos clientes  |
| **2** | Top 80% dos clientes  |
| **1** | 20% restantes         |

Essa pontuação nos ajuda a distinguir e focar nos clientes que mais contribuem para a receita da **KAMI CO**.


In [4]:
# Análise de Valor
import numpy as np
import plotly.graph_objects as go
def calculate_cutoffs(rfv_class_df):
  """
  Calculate cutoff ticket values for each value score based on 20% bands.
  """
  rfv_class_df = rfv_class_df[rfv_class_df['ticket_medio'] > 0]
  rfv_class_df_sorted = rfv_class_df.sort_values(by='ticket_medio', ascending=False)  
  cutoff_positions = [int((len(rfv_class_df)-1) * i / 5) for i in range(1, 6)]  
  cutoffs = {
    5: rfv_class_df_sorted.iloc[cutoff_positions[0]]['ticket_medio'],
    4: rfv_class_df_sorted.iloc[cutoff_positions[1]]['ticket_medio'],
    3: rfv_class_df_sorted.iloc[cutoff_positions[2]]['ticket_medio'],
    2: rfv_class_df_sorted.iloc[cutoff_positions[3]]['ticket_medio'],
    1: rfv_class_df_sorted.iloc[cutoff_positions[4]]['ticket_medio']
  }
  return cutoffs

def assign_value_score(ticket, cutoffs):
  """
  Assign value score based on the ticket average using cutoff values.
  """
  for score, cutoff in cutoffs.items():
    if ticket >= cutoff:
      return score
  return 1

def compute_value_scores(rfv_class_df):
  """
  Compute the 'value' scores for each customer.
  """
  cutoffs = calculate_cutoffs(rfv_class_df)  
  rfv_class_df['valor'] = rfv_class_df['ticket_medio'].apply(lambda x: assign_value_score(x, cutoffs))
  return rfv_class_df

highest_avg = rfv_class_df[rfv_class_df['ticket_medio'] == rfv_class_df['ticket_medio'].max()].iloc[0]
general_avg = rfv_class_df['ticket_medio'].mean()
def show_avg_ticket_analisys_indicator(df):
  top1_costumer_value_fig = go.Indicator(
    mode="number",
    title={
      'text': f"<span style='font-size:100%; color: steelgreen;'>Ticket Medio em 2022</span><br>",
      'font': {'size': 20}
    },
    value=highest_avg['ticket_medio'],
    number={'font': {'size': 24, 'color':"seagreen"}, 'prefix': "R$ "},
    domain={'row': 0, 'column': 0}
  )
  fig = go.Figure()
  fig.add_trace(top1_costumer_value_fig)
  fig.update_layout(
    title={
      'text': f"Cliente de Maior Valor em 2022<br>{highest_avg['nome_cliente']}<br>{highest_avg['razao_social']}",
      'y':0.9,
      'x':0.5,
      'xanchor': 'center',
      'yanchor': 'top'
    },
    grid={'rows': 1, 'columns': 1, 'pattern': "independent"},
    height=250,
    width=500
  )
  fig.show()  

def top_20_avg_ticket_graph(df):
  top_20 = rfv_class_df.nlargest(20, 'ticket_medio')
  top_20 = top_20.sort_values(by='ticket_medio', ascending=True)
  
  bar_fig = go.Figure(go.Bar(
    y=top_20['nome_cliente'],
    x=top_20['ticket_medio'],
    orientation='h',
    marker={
        'color': top_20['ticket_medio'],
        'colorscale': 'greens'
    },
    text=top_20['ticket_medio'],
    textposition='outside'
  ))
  
  avg_ticket_avg = top_20['ticket_medio'].mean()  

  bar_fig.update_layout(
    title="Top 20 Clientes por Ticket Medio em 2022",
    xaxis_title="Ticket Medio",
    yaxis_title="Cliente",
    showlegend=False,
    height=900,
    annotations=[
      {
        'x': avg_ticket_avg,
        'y': top_20['nome_cliente'].iloc[-1],
        'xref': 'x',
        'yref': 'y',
        'text': f"Média: R$ {format(avg_ticket_avg, ',.2f')}",
        'showarrow': True,
        'arrowhead': 4,
        'ax': 0,
        'ay': -40,
        'font': {'color': 'red', 'size': 18}
      }
    ]
  )
  bar_fig.show()
  
def show_value_legend(cutoffs):
  colors = ['#00441b', '#006d2c', '#238b45', '#41ab5d', '#74c476']
  formatted_values = ["R$ " + format(value, ',.2f') for value in cutoffs.values()]
  fig = go.Figure(data=[go.Table(
    header=dict(
      values=['Pontuação', 'Ticket Médio'],
      fill_color='darkgreen',
      font=dict(color='white', size=16),
      height=32
    ),
    cells=dict(
      values=[list(cutoffs.keys()), formatted_values],
      fill_color=[colors, colors],
      align=['center', 'center'],
      font=dict(color='black', size=14),
      height=32
    )
  )])
  fig.update_layout(
    title='Tabela de Referência para Pontuação de Valor',
    title_x=0.5,
    title_font_size=20,
    showlegend=False,
    width=800,
    height=400
  )
  fig.show()

def show_value_graphs(rfv_class_df): 
  rfv_class_df = compute_value_scores(rfv_class_df)
  abs_counts = rfv_class_df['valor'].value_counts().sort_index()
  perc_counts = (rfv_class_df['valor'].value_counts(normalize=True) * 100).sort_index()
  fig = sp.make_subplots(
    rows=1, cols=2,
    subplot_titles=("Clientes por Pontuação de Valor", "% de Clientes por Valor")
  )
  fig.add_trace(
    go.Bar(
      x=abs_counts, y=abs_counts.index.astype(str), orientation='h',
      marker=dict(color=abs_counts.values, colorscale='greens')
    ), 
    row=1, col=1
  )
  fig.update_xaxes(title_text="Número de Clientes (Unidades)", row=1, col=1)
  fig.update_yaxes(title_text="Pontuação de Valor", row=1, col=1)
  fig.add_trace(
    go.Bar(
      x=perc_counts.index.astype(str), y=perc_counts,
      marker=dict(color=perc_counts.values, colorscale='greens')
    ),
    row=1, col=2
  )
  fig.update_xaxes(title_text="Pontuação de Valor", row=1, col=2)
  fig.update_yaxes(title_text="% de Clientes (%)", row=1, col=2)  
  for i in fig['layout']['annotations']:
      i['font']['size'] = 20  
  fig.update_layout(showlegend=False, height=400)
  fig.show()

def show_value_indicators(df):
  green_colors = ["#AEDFA3", "#70C466", "#4DA94C", "#2A8F32", "#006D21"]
  avg_ticket_last_year = df['ticket_medio'].mean()
  avg_value = rfv_class_df['valor'].mean()
  target_top_20_ticket_avg = df.head(20)['ticket_medio'].mean()
  formatted_target = '{:,.2f}'.format(target_top_20_ticket_avg)
  avg_ticket_avg_trace = go.Indicator(
    mode='number+delta',
    title={
      'text': f"Média de Ticket Médio no Último Ano",
      'font': {'size': 22, 'color':green_colors[2]}
    },
    value=avg_ticket_last_year,
    number={
      'font': {'size': 50, 'color': green_colors[4]},
      'prefix': "R$ ",
      'valueformat': ',.2f'
    },
    delta={
      'relative': True,
      'reference': target_top_20_ticket_avg,
      'position': "bottom",
      'suffix': f" abaixo da meta de R$ {formatted_target} de ticket médio" if avg_ticket_last_year < target_top_20_ticket_avg else f' acima da meta de R$ {formatted_target} de ticket médio'
    },
    domain={'row': 0, 'column': 0}
  )
  avg_value_trace = go.Indicator(    
    mode='gauge+number',
    value=avg_value,
    gauge={
      'shape': "angular",
      'bar': {'color': "darkred"},
      'axis': {'range': [None, 5], 'tickvals': [1,2,3,4,5], 'ticktext': ["1", "2", "3", "4", "5"]},
      'steps': [
        {'range': [0, 1], 'color': green_colors[0]},
        {'range': [1, 2], 'color': green_colors[1]},
        {'range': [2, 3], 'color': green_colors[2]},
        {'range': [3, 4], 'color': green_colors[3]},
        {'range': [4, 5], 'color': green_colors[4]},
      ],
      'threshold': {
        'line': {'color': "darkred", 'width': 6},
        'thickness': 0.75,
        'value': avg_value
      }
    },
    title={
      'text': "Média de Valor",
      'font': {'size': 22, 'color': green_colors[2]}
    },
    number={'font': {'size': 80, 'color': green_colors[4]}},
    domain={'row': 0, 'column': 1}
  )
  fig = go.Figure()
  fig.add_trace(avg_ticket_avg_trace)
  fig.add_trace(avg_value_trace)
  fig.update_layout(grid={'rows': 1, 'columns': 2, 'pattern': "independent"}, height=600)
  fig.show()
rfv_class_df = rfv_class_df[rfv_class_df['ticket_medio'] > 0]
rfv_class_df = rfv_class_df.sort_values(by='ticket_medio', ascending=False)
show_avg_ticket_analisys_indicator(rfv_class_df)
top_20_avg_ticket_graph(rfv_class_df)
show_value_legend(cutoffs=calculate_cutoffs(rfv_class_df))
show_value_graphs(rfv_class_df)
show_value_indicators(rfv_class_df)
